# Data Science - Python and Pandas


## Table of Content

1. [Introduction](#introduction)<br>
1.1. [Series and DataFrames](#series)<br>
1.2. [Indexing](#index)<br>
1.3. [Data Selection](#selection)<br>
2. [Transform data](#transform)<br>
2.1. [Adding and deleting columns](#columns)<br>
2.2. [Missing Data](#nans)<br>
2.3. [Cleaning Data](#cleaning)<br>
2.4. [Merging Data](#merging)<br>
2.5. [Grouping Data](#grouping)<br>
3. [Visualise data](#visualise)<br>
4. [Excercises](#excercises)<br>



Let's start with loading the packages and a quick look at some data. Select the below cell by clicking on it, and then click on the `Run` button at the top of the notebook (or use `Shift+Enter`). This is how you can run all code cells in this notebook. The numbers in front of the cells tell you in which order you have run them, for instance `[1]`. When you see a `[*]` the cell is currently running and `[]` means you have not run the cell yet. 

In [ ]:
%matplotlib inline

import numpy as np
import pandas as pd

**ADD description on how download file, add to COS in github repo**

**ADD how to insert a file from COS here** Data from: https://github.com/the-pudding/data/tree/master/pockets

In [ ]:
jeans = pd.read_csv('jeans_measurements.csv')

Use `?` to find out what the options are for a command. For `read_csv` there are more than 50:

In [ ]:
?pd.read_csv

Now let's have a look at the data that has been loaded into the notebook. Use `jeans`, `jeans.head()` or `jeans.tail()` to see different parts of the table and `jeans.dtypes` to see which variables there are and what datatype they have. Add a number between the brackets `()` to specify how many lines you want to display.

> _Tip_: If you want to run these in separate cells, activate the below cell by clicking on it and then click on the `+` at the top of the notebook. This will add extra cells. Click on the upwards and downwards arrows to move the cells up and down to change to order. 

In [ ]:
jeans

<div class="alert alert-success">
 <b>EXERCISE</b> <br/> 
 Explore the following commands:
  <ul>
  <li><font face="Courier">jeans.columns</font></li>
  <li><font face="Courier">jeans.values</font></li>
  <li><font face="Courier">jeans.shape</font></li>
  <li><font face="Courier">len(jeans)</font></li>
  
 </ul> 
</div>  

In [ ]:
# try the commands here:


<a id="introduction"></a>
## 1. Introduction

The package we used to read this file and look at some of it's properties is [Pandas](https://pandas.pydata.org/), which is an open source library with easy-to-use data structures and data analysis tools. 

<div class="alert alert-info" style="font-size:100%">
<b>Read this <a href="http://pandas.pydata.org/pandas-docs/stable/getting_started/10min.html">10 minute introduction</a> for a quick overview of Pandas.<br>
</div>

<a id="series"></a>
### 1.1 Series and DataFrames 

Let's go through some of the basics of Pandas before going back to the Jeans dataset. Pandas has two main data structures: `Series` and `DataFrames`. 

A `Series` is a list of values with an integer index. The first column is the index (the default starts at 0) and the second column the values.

In [ ]:
s = pd.Series([1, 3, 5, np.nan, 6, 8])
s

 A `DataFrame` is similar, but has multiple columns. You can create one in many ways, by loading a file or from for example a NumPy array and a date for the index. (We come back to the index and dates later) 


<div class="alert alert-info" style="font-size:100%">
<b>Read this <a href="https://docs.scipy.org/doc/numpy-1.15.0/user/quickstart.html"> tutorial</a> for an overview of NumPy.<br>
</div>

Two examples:

In [ ]:
dates = pd.date_range('20130101', periods=6)
dates

In [ ]:
numbers = np.random.randn(6, 4)
numbers

In [ ]:
df = pd.DataFrame(numbers, index=dates, columns=list('ABCD'))
df

In [ ]:
df2 = pd.DataFrame({'A': 1.,
                     'B': pd.Timestamp('20130102'),
                     'C': pd.Series(1, index=list(range(4)), dtype='float32'),
                     'D': np.array([3] * 4, dtype='int32'),
                     'E': pd.Categorical(["test", "train", "test", "train"]),
                     'F': 'foo'})

In [ ]:
df2.head()

To find out what the data type is of a variable use `type()`: 

In [ ]:
print('Data type of s is '+str(type(s)))
print('Data type of s is '+str(type(dates)))
print('Data type of s is '+str(type(numbers)))
print('Data type of df is '+str(type(df)))

In [ ]:
type(jeans)

<a id="index"></a>
### 1.2 Indexing 

It is important to understand the `index` to work with dataframes, so let's explore this a little further. 

For this we will create a new DataFrame with the population of the 5 largst cities in the UK ([source](https://en.wikipedia.org/wiki/List_of_urban_areas_in_the_United_Kingdom)). `data` is a [dictionary](https://realpython.com/python-dicts/).

In [ ]:
data = {'city':       ['London','Manchester','Birmingham','Leeds','Glasgow'],
        'population': [9787426,  2553379,     2440986,    1777934,1209143],
        'area':       [1737.9,   630.3,       598.9,      487.8,  368.5 ]}
cities = pd.DataFrame(data)
cities

In [ ]:
cities.index

In [ ]:
cities.columns

In [ ]:
cities = cities.set_index('city')
cities

In [ ]:
cities = cities.reset_index()
cities = cities.set_index(['city','population'])
cities

In [ ]:
cities = cities.reset_index()
cities = cities.set_index('city')
cities

<a id="selection"></a>
### 1.3 Data Selection

Select a single number or a range of numbers by label with `.loc[]` (This only works for the column that was set to the index):

In [ ]:
cities.loc['London', 'area']

In [ ]:
cities.loc['Manchester':'Leeds', ['area', 'population']]

Or select by position with `.iloc[]`. You can select a single row, multiple rows (or columns) at particular positions in the index, it only takes integers:

In [ ]:
cities.iloc[0]

In [ ]:
cities.iloc[:,1]

In [ ]:
cities.iloc[:,0:2]

In [ ]:
cities.iloc[2:4,0:2]

You can also use one or more column names to create a new DataFrame.

In [ ]:
cities['area']

In [ ]:
cities2 = cities[['area','population']]
cities2

#### Filtering

Selecting rows based on a certain condition can be done with Boolean indexing:

In [ ]:
cities['area'] > 500

In [ ]:
cities[cities['area'] > 500]

In [ ]:
cities[(cities['area'] > 500) & (cities['population'] > 2500000)]

In [ ]:
cities[(cities['area'] < 500) | (cities['population'] < 1000000)]

In [ ]:
cities[cities['area'] == 487.8] 

<div class="alert alert-success">
 <b>EXERCISE</b> <br/> 
 Using the jeans DataFrame:
  <ul>
  <li>Find the most expensive jeans</li>  
  <li>Find the cheapest jeans</li>
  <li>What is the difference in price between the cheapest and most expensive jeans?</li>    
 </ul>  
</div>  

> *Tips*: 
- Find the maximum of a row with for instance `jeans['price'].max()` 
- Extract the value from a cell in a DataFrame with `.value[]`
- To see the answer uncomment the line in the cells that contain %load (remove the `#`) and run the cell, but try to find your own solution first in the cell above the solution!

In [ ]:
# most expensive jeans


In [ ]:
# %load https://raw.githubusercontent.com/IBMDeveloperUK/pandas-workshop/master/answers/answer1.py
jeans[jeans['price'] == jeans['price'].max()]


In [ ]:
# cheapest jeans


In [ ]:
# %load https://raw.githubusercontent.com/IBMDeveloperUK/pandas-workshop/master/answers/answer2.py

In [ ]:
# difference in price
expensive = (jeans['price'][jeans['price'] == jeans['price'].max()]).values[0]
cheap = (jeans['price'][jeans['price'] == jeans['price'].min()]).values[0]

expensive - cheap

In [ ]:
# %load https://raw.githubusercontent.com/IBMDeveloperUK/pandas-workshop/master/answers/answer3.py

<a id="transform"></a>
## 2. Transform Data

When looking at date there are always transformations needed to get it in the format you need for your anlysis, visulasations and machine learning models. This is an important skill to master!

These are a few examples of the endless possibilities. The best way to learn is to find a dataset and try to answer questions that you could answer with the data. 

<a id="columns"></a>
### 2.1 Adding and deleting columns
Adding a column can be done by defining a new column, which can then be dropped with 'drop'. 

In [ ]:
jeans['new'] = 1
jeans = jeans.drop(columns=['new'])

In [ ]:
jeans['avgHeightFront'] = (jeans.maxHeightFront + jeans.minHeightFront) / 2

In [ ]:
jeans.head()

<a id="nans"></a>
### 2.2 Missing Data

As there is no missing data in the jeans dataset let's create a copy of the data and add some. You have to add `.copy()` else all changes made to `jeans2` will also be made to `jeans`. 

In [ ]:
jeans2 = jeans.copy()

In [ ]:
jeans2.iloc[[2,3,9],5] = 'NaN'
jeans2.iloc[[44,2,11],9] = 'NaN'
jeans2.iloc[[5,12,13,22,44,45],16] = 'NaN'

There are several ways to handle these missing values. Delete the rows that contain them:

In [ ]:
Fill them with for instance the average of the column, or any other value

<a id="cleaning"></a>
### 2.3 Cleaning Data


In [ ]:
# menWomen to two columns with 0 and 1

In [ ]:
# fabric to multiple columns

<a id="merging"></a>
### 2.4 Merging Data


In [ ]:
# merge data

<a id="grouping"></a>
### 2.5 Grouping Data


In [ ]:
# group data

<div class="alert alert-success">
 <b>EXERCISE</b> <br/> 
 Using the jeans DataFrame:
  <ul>
  <li>Add a column `FrontArea` with the area of the front pocket (height X width) </li>        
  <li>Add a column `BackArea` with the area of the back pocket (height X width) </li>        
  <li>How many styles of jeans are there?</li>        
  <li>How many men and women's jeans are measured? And what are the average front and back pocket sizes of each?</li>  
 </ul>  
</div>  

> *Tips*: 
- To find out how many unique values there are in a column use `np.unique(df['a'])`

In [ ]:
# FrontArea

In [ ]:
# BackArea

In [ ]:
# unique number of jeans styles
len(np.unique(jeans['style']))

<a id="explore"></a>
## 3. Visualizing Data

### Time Series

In [ ]:
# sort, groupBy

<div class="alert alert-success">
 <b>EXERCISE</b>
 <ul>
  <li>Are mens or womens jeans more expensive?</li>
  <li>Which brand has the most styles of jeans?</li>
  <li></li>
 </ul> 
 
</div>    